In [ ]:
!pip install fastaudio

In [ ]:
!pip install -U fastai
# If weird torch install error thing shows up, go into path, replace erroneous line with the following: from torch.hub import load_state_dict_from_url

In [ ]:
!pip show fastai

In [ ]:
!pip install pydub
!pip install moviepy

In [ ]:
# NO NEED TO INSTALL ANYTHING ELSE
import os
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

In [ ]:
PATH_TO_DATASET = "drive/MyDrive/Colab Notebooks/Jazz vs Classical Classifier/dataset"
CLASSICAL_CODE = 0
JAZZ_CODE = 1
LENGTH_OF_CLIP = 5000

fnames = get_files(PATH_TO_DATASET, extensions=".wav", recurse=True)

In [ ]:
classical_names = get_files(PATH_TO_DATASET + "/Classical", extensions=".wav", recurse=True)
print("Number of classical: " + str(len(classical_names)))
print("Number of jazz: " + str(len(fnames) - len(classical_names)))

In [ ]:
resized = 1000

cfg = AudioConfig.Voice()
aud2spec = AudioToSpec.from_cfg(cfg)
crop1s = ResizeSignal(resized)
pipe = Pipeline([AudioTensor.create, crop1s, aud2spec])

item_tfms = [ResizeSignal(resized), aud2spec]

def _get_labels(x):
    artists = ["bach", "beethoven", "handel", "mozart", "stravinasky",
               "tchaikovsky", "verdi", "wagner", "dvorak", "schubert", "brahms",
               "schumann", "mendelssohn", "mussorgsky", "chopin", "berlioz",
               "puccini", "liszt", "vivaldi", "strauss_r", "mahler_g", "sibelius_j",
               "faure", "rachmaninoff", "debussy", "grieg_e", "shostakovich",
               "copland_a", "bizet", "rimsky_korsakov", "palestrina", "bruckner_a",
               "bartok"]
    for artist in artists:
        if artist in str(x):
            return CLASSICAL_CODE
    return JAZZ_CODE

sounds = DataBlock(blocks=(AudioBlock, CategoryBlock),
                 get_items=get_audio_files,
                 splitter=RandomSplitter(),
                 item_tfms = item_tfms,
                 get_y=_get_labels)
dls = sounds.dataloaders(PATH_TO_DATASET, bs=64)
dls.valid.vocab

/usr/local/lib/python3.10/dist-packages/fastai/torch_core.py:263: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  return getattr(torch, 'has_mps', False)


[0, 1]

In [ ]:
def alter_learner(learn, n_channels=1):
  "Adjust a `Learner`'s model to accept `1` channel"
  layer = learn.model[0][0]
  layer.in_channels=n_channels
  layer.weight = nn.Parameter(layer.weight[:,1,:,:].unsqueeze(1))
  learn.model[0][0] = layer

In [ ]:
learn = Learner(dls, xresnet18(), metrics=accuracy)
alter_learner(learn, 1)

In [ ]:
lr = learn.lr_find() # SuggestedLRs(valley=0.0010000000474974513)
lr

SuggestedLRs(valley=4.365158383734524e-05)


In [ ]:
learn.fit_one_cycle(6, 4e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.420809,2.169612,0.654412,21:36
1,0.318778,0.365566,0.866516,19:01
2,0.245025,0.235790,0.909502,18:42
3,0.197160,0.373785,0.857466,18:48
4,0.152693,0.145310,0.941742,18:25
5,0.128353,0.134052,0.951923,18:06


In [ ]:
learn.export()

In [ ]:
learn_inf = load_learner('export.pkl')

In [ ]:
model = learn_inf.model
type(model)

fastai.vision.models.xresnet.XResNet

In [ ]:
def my_prediction(path):
    files = os.listdir(path)
    results = 0
    for file in files:
      temp_path = path + file
      pred = int(learn_inf.predict(Path(temp_path))[0])
      #print(pred)
      results += pred
    percentage = 100 - (results * 100 // len(files)) if results < len(files) // 2 else results * 100 // len(files)
    print("For " + str(files[0]) + "...")
    print("Confidence: " + str(percentage) + "%")
    return "Jazz" if results > len(files) // 2 else "Classical"

In [ ]:
def visualize(x,y):
    if isinstance(x, torch.Tensor):
        x = x.tolist()
    if isinstance(y, torch.Tensor):
        y = y.tolist()
    list = x[0] + x[1]
    plt.plot(np.array(list))
    if (y == 1):
        plt.xlabel("Classical clip")
    else:
        plt.xlabel("Jazz clip")
    plt.show()